In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
from skimage import io
import seaborn as sns
import warnings
import numpy as np
import warnings
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pylab import mpl, plt
import matplotlib.patches as mpatches
from tqdm.notebook import tqdm

# best font and style settings for notebook 
warnings.filterwarnings('ignore')
sns.set_style("white")
mpl.rcParams['font.family'] = 'MiSans'

model_path = r"D:\pythonProject\DeepSeek\Recsys\AnimeLLMRec\Qwen3-0.6B"  # modify to your Qwen Path

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
from delta_trainer import train_delta_from_H, generate_by_H, evaluate_slot_ceval, evaluate_slot_ceval_eos, \
    evaluate_slot_ceval_eos_2

# 构造 prompt & 得到 H_state
prompt = "请写一段关于AI教育的引言。"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True, return_dict=True)
H = outputs.hidden_states[-1]

# 调用 delta 训练
delta_3 = train_delta_from_H(model, tokenizer, prompt, H, step=3)
delta_10 = train_delta_from_H(model, tokenizer, prompt, H, step=10)
delta_30 = train_delta_from_H(model, tokenizer, prompt, H, step=30)


In [16]:
# generate_by_H(model=model, prompt=prompt, tokenizer=tokenizer, delta=delta_3, answer_len=200)

In [17]:
from datasets import get_dataset_config_names

# 获取本地路径 "./ceval-exam" 中可用的所有子数据集名称（config names）
dataset_path = "./ceval-exam"
dataset_names = get_dataset_config_names(path=dataset_path)
dataset_names

['accountant',
 'advanced_mathematics',
 'art_studies',
 'basic_medicine',
 'business_administration',
 'chinese_language_and_literature',
 'civil_servant',
 'clinical_medicine',
 'college_chemistry',
 'college_economics',
 'college_physics',
 'college_programming',
 'computer_architecture',
 'computer_network',
 'discrete_mathematics',
 'education_science',
 'electrical_engineer',
 'environmental_impact_assessment_engineer',
 'fire_engineer',
 'high_school_biology',
 'high_school_chemistry',
 'high_school_chinese',
 'high_school_geography',
 'high_school_history',
 'high_school_mathematics',
 'high_school_physics',
 'high_school_politics',
 'ideological_and_moral_cultivation',
 'law',
 'legal_professional',
 'logic',
 'mao_zedong_thought',
 'marxism',
 'metrology_engineer',
 'middle_school_biology',
 'middle_school_chemistry',
 'middle_school_geography',
 'middle_school_history',
 'middle_school_mathematics',
 'middle_school_physics',
 'middle_school_politics',
 'modern_chinese_histor

In [18]:
from datasets import load_dataset

dataset = load_dataset(r"./ceval-exam", name="computer_network")
print(dataset['val'][0])

{'id': 0, 'question': '使用位填充方法，以01111110为位首flag，数据为011011111111111111110010，求问传送时要添加几个0____', 'A': '1', 'B': '2', 'C': '3', 'D': '4', 'answer': 'C', 'explanation': ''}


In [19]:
def evaluate_slot_ceval_eos(model, tokenizer, delta, example, max_len=20, verbose=True):
    """
    基于 generate_by_H_eos 的评估函数，用于 C-Eval 单选题目。

    返回：
    - predict_option: 预测选项，如 'A'
    - is_correct: 是否预测正确
    """
    prompt = f"""以下是一道单项选择题，请你阅读题目并选择最合适的选项。

题目：{example['question']}

选项：
A. {example['A']}
B. {example['B']}
C. {example['C']}
D. {example['D']}

答案是："""

    output_text = generate_by_H_eos(model, prompt, tokenizer, delta, answer_len=max_len)

    if verbose:
        print("🔍 模型生成结果:\n", output_text)

    predict_option = None
    for option in ['A', 'B', 'C', 'D']:
        if option in output_text:
            predict_option = option
            break

    is_correct = (predict_option == example['answer'])
    # return predict_option, is_correct
    return output_text, predict_option, example['answer'], is_correct

In [20]:
# dataset_name = "computer_network"
# dataset = load_dataset(r"./ceval-exam", name=dataset_name)
# 
# correct = 0
# total = 0
# 
# answer_sheet = []
# for ex in tqdm(dataset['val'], desc="Evaluating per-question delta"):
#     # === 构造每道题的 Prompt ===
#     prompt = f"""以下是一道单项选择题，请你阅读题目，结合题目的知识背景，选择最合适的选项。
#     题目：{ex['question']}
#     
#     选项：
#     A. {ex['A']}
#     B. {ex['B']}
#     C. {ex['C']}
#     D. {ex['D']}
#     
#     答案是："""
# 
#     # === 获取 H_state ===
#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#     with torch.no_grad():
#         outputs = model(**inputs, output_hidden_states=True, return_dict=True)
#     H = outputs.hidden_states[-1]
# 
#     # === 训练 delta（例如3步）===
#     delta = train_delta_from_H(model, tokenizer, prompt, H, step=30)
# 
#     # === 推理与评估 ===
#     pred, pre_answer, answer, is_correct = evaluate_slot_ceval_eos(model, tokenizer, delta, ex, max_len=20,
#                                                                    verbose=False)
#     correct += int(is_correct)
#     total += 1
#     answer_sheet.append([pred, pre_answer, answer, is_correct, dataset_name])
# print(f"🎯 Accuracy (per-question delta): {correct}/{total} = {correct / total:.2%}")


In [25]:
def evaluate_slot_ceval_eos(model, tokenizer, delta, example, prompt, max_len=20, verbose=True):
    """
    基于 generate_by_H_eos 的评估函数，用于 C-Eval 单选题目。

    返回：
    - predict_option: 预测选项，如 'A'
    - is_correct: 是否预测正确
    # """

    output_text = generate_by_H_eos(model, prompt, tokenizer, delta, answer_len=max_len)

    if verbose:
        print("🔍 模型生成结果:\n", output_text)

    predict_option = None
    for option in ['A', 'B', 'C', 'D']:
        if option in output_text:
            predict_option = option
            break

    is_correct = (predict_option == example['answer'])
    # return predict_option, is_correct
    return output_text, predict_option, example['answer'], is_correct


def eval_dataset(dataset_name, step=3, max_len=50):
    # dataset_name = "computer_network"
    dataset = load_dataset(r"./ceval-exam", name=dataset_name)

    correct = 0
    total = 0

    answer_sheet = []
    for ex in dataset['val']:
        # === 构造每道题的 Prompt ===
        prompt = f"""以下是一道单项选择题，请你阅读题目，选择最合适的选项。
        题目：{ex['question']}
        选项：
        A. {ex['A']}
        B. {ex['B']}
        C. {ex['C']}
        D. {ex['D']}
        答案是："""
        prompt = f"""请你扮演一位专业的考试助手，阅读下面的单项选择题，并根据内容在四个选项中选出最合适的一个。
        
        题目：
        {ex['question']}
        
        选项：
        A. {ex['A']}
        B. {ex['B']}
        C. {ex['C']}
        D. {ex['D']}
        
        请直接回答选项字母（A/B/C/D）。
        答案是："""

        # === 获取 H_state ===
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True, return_dict=True)
        H = outputs.hidden_states[-1]

        # === 训练 delta（例如3步）===
        delta = train_delta_from_H(model, tokenizer, prompt, H, step=step, lr=1e-1)

        # === 推理与评估 ===
        pred_txt, pre_answer, answer, is_correct = evaluate_slot_ceval_eos(model=model, tokenizer=tokenizer,
                                                                           delta=delta,
                                                                           example=ex, max_len=max_len, prompt=prompt,
                                                                           verbose=False)
        correct += int(is_correct)
        total += 1
        answer_sheet.append([prompt, pred_txt, pre_answer, answer, is_correct, dataset_name])
    print(f"🎯 {dataset_name} Accuracy (per-question delta): {correct}/{total} = {correct / total:.2%}")
    return answer_sheet

In [22]:
from delta_trainer import generate_by_H_eos



In [23]:
len(dataset_names)

52

In [26]:
step = 1
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

  0%|          | 0/52 [00:00<?, ?it/s]

🎯 accountant Accuracy (per-question delta): 21/49 = 42.86%
🎯 advanced_mathematics Accuracy (per-question delta): 5/19 = 26.32%
🎯 art_studies Accuracy (per-question delta): 18/33 = 54.55%
🎯 basic_medicine Accuracy (per-question delta): 13/19 = 68.42%
🎯 business_administration Accuracy (per-question delta): 15/33 = 45.45%
🎯 chinese_language_and_literature Accuracy (per-question delta): 12/23 = 52.17%
🎯 civil_servant Accuracy (per-question delta): 17/47 = 36.17%
🎯 clinical_medicine Accuracy (per-question delta): 11/22 = 50.00%
🎯 college_chemistry Accuracy (per-question delta): 12/24 = 50.00%



KeyboardInterrupt



In [27]:
step = 2
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

  0%|          | 0/52 [00:00<?, ?it/s]

🎯 accountant Accuracy (per-question delta): 22/49 = 44.90%
🎯 advanced_mathematics Accuracy (per-question delta): 4/19 = 21.05%
🎯 art_studies Accuracy (per-question delta): 15/33 = 45.45%
🎯 basic_medicine Accuracy (per-question delta): 12/19 = 63.16%
🎯 business_administration Accuracy (per-question delta): 16/33 = 48.48%
🎯 chinese_language_and_literature Accuracy (per-question delta): 12/23 = 52.17%
🎯 civil_servant Accuracy (per-question delta): 18/47 = 38.30%
🎯 clinical_medicine Accuracy (per-question delta): 11/22 = 50.00%
🎯 college_chemistry Accuracy (per-question delta): 11/24 = 45.83%
🎯 college_economics Accuracy (per-question delta): 27/55 = 49.09%
🎯 college_physics Accuracy (per-question delta): 11/19 = 57.89%
🎯 college_programming Accuracy (per-question delta): 16/37 = 43.24%
🎯 computer_architecture Accuracy (per-question delta): 7/21 = 33.33%
🎯 computer_network Accuracy (per-question delta): 7/19 = 36.84%



KeyboardInterrupt



In [29]:
step = 0
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

  0%|          | 0/52 [00:00<?, ?it/s]

🎯 accountant Accuracy (per-question delta): 22/49 = 44.90%
🎯 advanced_mathematics Accuracy (per-question delta): 4/19 = 21.05%
🎯 art_studies Accuracy (per-question delta): 18/33 = 54.55%
🎯 basic_medicine Accuracy (per-question delta): 13/19 = 68.42%
🎯 business_administration Accuracy (per-question delta): 16/33 = 48.48%
🎯 chinese_language_and_literature Accuracy (per-question delta): 11/23 = 47.83%
🎯 civil_servant Accuracy (per-question delta): 19/47 = 40.43%
🎯 clinical_medicine Accuracy (per-question delta): 12/22 = 54.55%
🎯 college_chemistry Accuracy (per-question delta): 10/24 = 41.67%
🎯 college_economics Accuracy (per-question delta): 26/55 = 47.27%
🎯 college_physics Accuracy (per-question delta): 11/19 = 57.89%
🎯 college_programming Accuracy (per-question delta): 18/37 = 48.65%
🎯 computer_architecture Accuracy (per-question delta): 10/21 = 47.62%
🎯 computer_network Accuracy (per-question delta): 7/19 = 36.84%
🎯 discrete_mathematics Accuracy (per-question delta): 5/16 = 31.25%
🎯 ed


KeyboardInterrupt



In [ ]:
step = 0
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
step = 1
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [11]:
step = 3
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

  0%|          | 0/52 [00:00<?, ?it/s]

🎯 accountant Accuracy (per-question delta): 22/49 = 44.90%
🎯 advanced_mathematics Accuracy (per-question delta): 4/19 = 21.05%
🎯 art_studies Accuracy (per-question delta): 18/33 = 54.55%
🎯 basic_medicine Accuracy (per-question delta): 13/19 = 68.42%
🎯 business_administration Accuracy (per-question delta): 16/33 = 48.48%
🎯 chinese_language_and_literature Accuracy (per-question delta): 11/23 = 47.83%
🎯 civil_servant Accuracy (per-question delta): 19/47 = 40.43%
🎯 clinical_medicine Accuracy (per-question delta): 12/22 = 54.55%
🎯 college_chemistry Accuracy (per-question delta): 10/24 = 41.67%
🎯 college_economics Accuracy (per-question delta): 26/55 = 47.27%



KeyboardInterrupt



In [12]:
step = 10
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

  0%|          | 0/52 [00:00<?, ?it/s]

🎯 accountant Accuracy (per-question delta): 20/49 = 40.82%
🎯 advanced_mathematics Accuracy (per-question delta): 4/19 = 21.05%
🎯 art_studies Accuracy (per-question delta): 18/33 = 54.55%
🎯 basic_medicine Accuracy (per-question delta): 13/19 = 68.42%
🎯 business_administration Accuracy (per-question delta): 16/33 = 48.48%
🎯 chinese_language_and_literature Accuracy (per-question delta): 11/23 = 47.83%
🎯 civil_servant Accuracy (per-question delta): 19/47 = 40.43%
🎯 clinical_medicine Accuracy (per-question delta): 12/22 = 54.55%
🎯 college_chemistry Accuracy (per-question delta): 10/24 = 41.67%
🎯 college_economics Accuracy (per-question delta): 26/55 = 47.27%


KeyboardInterrupt: 

In [13]:
step = 100
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

  0%|          | 0/52 [00:00<?, ?it/s]

🎯 accountant Accuracy (per-question delta): 22/49 = 44.90%
🎯 advanced_mathematics Accuracy (per-question delta): 4/19 = 21.05%


KeyboardInterrupt: 

In [ ]:
step = 20
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
step = 3
max_len = 20
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
dataset_names

In [ ]:
step = 10
max_len = 20
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
step = 20
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
step = 0
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
step = 3
max_len = 10
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
step = 5
max_len = 20
answer_sheet = []
for i in tqdm(dataset_names[:5]):
    answer_sheet += eval_dataset(i, step=step, max_len=max_len)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
step = 6
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
step = 30
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
step = 0
answer_sheet = []
for i in tqdm(dataset_names[:]):
    answer_sheet += eval_dataset(i, step=step)
    df_answer = pd.DataFrame(answer_sheet)
    df_answer.to_csv(f"./eval_result/answer_step_{step}.csv", index=False)

In [ ]:
df_answer

In [ ]:
pd.DataFrame(answer_sheet)

In [ ]:
# from tqdm import tqdm
from tqdm.notebook import tqdm

correct = 0
total = 0

for ex in tqdm(dataset['test'], desc="Evaluating per-question delta"):
    # === 构造每道题的 Prompt ===
    prompt = f"""以下是一道单项选择题，请你阅读题目并选择最合适的选项。

题目：{ex['question']}

选项：
A. {ex['A']}
B. {ex['B']}
C. {ex['C']}
D. {ex['D']}

答案是："""

    # === 获取 H_state ===
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True, return_dict=True)
    H = outputs.hidden_states[-1]

    # === 训练 delta（例如3步）===
    delta = train_delta_from_H(model, tokenizer, prompt, H, step=3)

    # === 推理与评估 ===
    pred, is_correct = evaluate_slot_ceval_eos(model, tokenizer, delta, ex, max_len=200, verbose=False)
    print(pred, ex)
    correct += int(is_correct)
    total += 1

print(f"🎯 Accuracy (per-question delta): {correct}/{total} = {correct / total:.2%}")


In [ ]:
def get_empty_delta(H_state):
    hidden_size = H_state.size(-1)
    delta = nn.Parameter(torch.zeros((1, 1, hidden_size), device=H_state.device, requires_grad=True))

    return delta


delta_empty = get_empty_delta(H_state=H)
delta_empty.shape

In [ ]:
from tqdm.notebook import tqdm

correct = 0
total = 0

for ex in tqdm(dataset['test'], desc="Evaluating per-question delta"):
    # === 构造每道题的 Prompt ===
    prompt = f"""以下是一道单项选择题，请你阅读题目并选择最合适的选项。
    
    题目：{ex['question']}
    
    选项：
    A. {ex['A']}
    B. {ex['B']}
    C. {ex['C']}
    D. {ex['D']}
    
    答案是："""

    # === 推理与评估 ===
    pred, is_correct = evaluate_slot_ceval_eos(model, tokenizer, delta_empty, ex, max_len=20, verbose=True)
    correct += int(is_correct)
    total += 1

print(f"🎯 Accuracy (per-question delta): {correct}/{total} = {correct / total:.2%}")


In [ ]:
from delta_trainer import generate_by_H_eos


def evaluate_slot_ceval_eos_2(model, tokenizer, delta, example, max_len=20, verbose=True):
    """
    基于 generate_by_H_eos 的评估函数，用于 C-Eval 单选题目。

    返回：
    - predict_option: 预测选项，如 'A'
    - is_correct: 是否预测正确
    """
    prompt = f"""以下是一道单项选择题，请你阅读题目并选择最合适的选项。

题目：{example['question']}

选项：
A. {example['A']}
B. {example['B']}
C. {example['C']}
D. {example['D']}

答案是："""

    output_text = generate_by_H_eos(model, prompt, tokenizer, delta, answer_len=max_len)

    if verbose:
        print("🔍 模型生成结果:\n", output_text)

    predict_option = None
    for option in ['A', 'B', 'C', 'D']:
        if option in output_text:
            predict_option = option
            break

    print(predict_option, example['answer'])
    is_correct = (predict_option == example['answer'])
    return predict_option, predict_option, example['answer']


In [ ]:
from tqdm.notebook import tqdm

correct = 0
total = 0

for ex in tqdm(dataset['test'], desc="Evaluating per-question delta"):
    # === 构造每道题的 Prompt ===
    prompt = f"""以下是一道单项选择题，请你阅读题目并选择最合适的选项。
    
    题目：{ex['question']}
    
    选项：
    A. {ex['A']}
    B. {ex['B']}
    C. {ex['C']}
    D. {ex['D']}
    
    答案是："""

    # === 推理与评估 ===
    pred, predict, truth = evaluate_slot_ceval_eos_2(model, tokenizer, delta_empty, ex, max_len=20, verbose=False)
    # print(predict,truth)
    correct += int(predict == truth)
    total += 1

print(f"🎯 Accuracy (per-question delta): {correct}/{total} = {correct / total:.2%}")


In [ ]:
for ex in tqdm(dataset['val'], desc="Evaluating per-question delta"):
    # === 构造每道题的 Prompt ===
    prompt = f"""以下是一道单项选择题，请你阅读题目并选择最合适的选项。

题目：{ex['question']}

选项：
A. {ex['A']}
B. {ex['B']}
C. {ex['C']}
D. {ex['D']}

答案是："""
    print(prompt)